# Lab 06 Softmax Zoo_classifier

* Softmax를 사용하여 분류를 진행합니다.
* 기존의 Version과변화가 없으므로 시즌1의 강의와 코드를 참조하시기 바랍니다.: [시즌1 lab-06-02 코드](https://github.com/hunkim/DeepLearningZeroToAll/blob/master/lab-06-2-softmax_zoo_classifier.py)

### 기본 Library 선언 및 Tensorflow 버전 확인

In [1]:
# Lab 6 Softmax Classifier
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

In [2]:
# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

(101, 16) (101, 1)


In [4]:
nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot:", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape one_hot:", Y_one_hot)

one_hot: Tensor("one_hot_1:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape_1:0", shape=(?, 7), dtype=float32)


In [6]:
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                 labels=tf.stop_gradient([Y_one_hot])))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y: y_data})
                                        
        if step % 100 == 0:
            print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%}".format(step, cost_val, acc_val))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Step:     0	Cost: 4.096	Acc: 6.93%
Step:   100	Cost: 0.825	Acc: 84.16%
Step:   200	Cost: 0.527	Acc: 90.10%
Step:   300	Cost: 0.396	Acc: 91.09%
Step:   400	Cost: 0.316	Acc: 94.06%
Step:   500	Cost: 0.262	Acc: 95.05%
Step:   600	Cost: 0.223	Acc: 97.03%
Step:   700	Cost: 0.192	Acc: 97.03%
Step:   800	Cost: 0.167	Acc: 97.03%
Step:   900	Cost: 0.146	Acc: 97.03%
Step:  1000	Cost: 0.130	Acc: 97.03%
Step:  1100	Cost: 0.116	Acc: 97.03%
Step:  1200	Cost: 0.104	Acc: 99.01%
Step:  1300	Cost: 0.095	Acc: 100.00%
Step:  1400	Cost: 0.087	Acc: 100.00%
Step:  1500	Cost: 0.081	Acc: 100.00%
Step:  1600	Cost: 0.075	Acc: 100.00%
Step:  1700	Cost: 0.070	Acc: 100.00%
Step:  1800	Cost: 0.066	Acc: 100.00%
Step:  1900	Cost: 0.062	Acc: 100.00%
Step:  2000	Cost: 0.058	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3